In [1]:
from neo4j import GraphDatabase

In [2]:
NEO4J_URI="bolt://3.95.184.198:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="networks-shelters-modules"

In [3]:
driver = GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
driver.verify_connectivity()

/tmp/ipykernel_71816/3490514894.py:2: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


'Neo4j/4.4.10'

In [16]:
# Unit of work
def get_actors(tx, movie): # (1)
    query = """
        MATCH (p:Person)
        /// MATCH (p:Person)-[:ACTED_IN]->(:Movie {title: $title})
        RETURN p LIMIT 10
    """
    result = tx.run(query, title=movie)
    print(result.peek())

    # Access the `p` value from each record
    return [ record["p"] for record in result ]

# # Open a Session
# with driver.session() as session:
#     # Run the unit of work within a Read Transaction
#     actors = session.read_transaction(get_actors, movie="The Green Mile") # (2)

#     for record in actors:
#         # print(record["p"])
#         print(record)

#     session.close()

In [18]:
# Open a Session
with driver.session() as session:
    # Run the unit of work within a Read Transaction
    actors = session.read_transaction(get_actors, movie="The Green Mile") # (2)

    for record in actors:
    
        # Check the first record without consuming it
        print(record.keys())
        # print(peek)
    session.close()

<Record p=<Node id=9816 labels=frozenset({'Actor', 'Person'}) properties={'bornIn': 'France', 'tmdbId': '1271225', 'imdbId': '2083046', 'born': neo4j.time.Date(1877, 2, 4), 'name': 'François Lallement', 'died': neo4j.time.Date(1954, 1, 1), 'url': 'https://themoviedb.org/person/1271225'}>>
dict_keys(['bornIn', 'tmdbId', 'imdbId', 'born', 'name', 'died', 'url'])
dict_keys(['tmdbId', 'imdbId', 'born', 'name', 'died', 'url'])
dict_keys(['bornIn', 'tmdbId', 'imdbId', 'born', 'name', 'bio', 'died', 'poster', 'url'])
dict_keys(['bornIn', 'tmdbId', 'imdbId', 'born', 'name', 'bio', 'died', 'poster', 'url'])
dict_keys(['bornIn', 'tmdbId', 'imdbId', 'born', 'name', 'bio', 'died', 'poster', 'url'])
dict_keys(['tmdbId', 'imdbId', 'born', 'name', 'bio', 'died', 'poster', 'url'])
dict_keys(['tmdbId', 'imdbId', 'name', 'url'])
dict_keys(['tmdbId', 'imdbId', 'name', 'url'])
dict_keys(['bornIn', 'tmdbId', 'imdbId', 'born', 'name', 'died', 'url'])
dict_keys(['bornIn', 'tmdbId', 'imdbId', 'born', 'name', 

In [19]:
def get_actors_consume(tx, name):
    result = tx.run("""
        MERGE (p:Person {name: $name})
        RETURN p
    """, name=name)

    info = result.consume()

In [21]:
get_actors_consume(name="François Lallement")

TypeError: get_actors_consume() missing 1 required positional argument: 'tx'